<a href="https://colab.research.google.com/github/Haheohohu/-/blob/main/%5BSQL_%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B6%84%EC%84%9D_%EA%B8%B0%EC%B4%88_7%EC%A3%BC%EC%B0%A8%5D_%EC%84%9C%EB%B9%84%EC%8A%A4_%ED%98%84%ED%99%A9_%EB%B6%84%EC%84%9D_A%ED%8C%80_%EA%B3%B5%EC%9C%A0%EC%9A%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 프로젝트 - 로컬팜 서비스 현황 분석



---



# 분석을 위한 기본 설정

## 기본 라이브러리 설정

In [ ]:
!pip install pandas folium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd 
import numpy as np
import folium

## Google Drive 마운트

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

## BigQuery 연동을 위한 기본 설정

In [ ]:
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table

project = 'crested-drive-372522' # Project ID inserted based on the query results selected to explore
location = 'US' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()
auth.authenticate_user()



---



# 로컬팜의 현재 상황 및 주요 이슈 

> **분석 진행에 앞서 수강생 분들께 전달되는 가상의 업체 상황**
- 21년 7월 목동의 유승상가에 픽업존 운영을 시작으로 현재는 총 8개의 매장을 운영중
- 22년 11월 서울지역 과일전문 다판다마켓을 인수(10월 말 부터 시범판매 시작, 11월 둘째주에 매장 오픈) 
- 물품을 소싱하는 판매자는 최초 (김지은-4f1d3d66, 양민석-34e92532, 손미경-afe78ffc) 3명으로 시작하였으나, 다판다마켓 인수와 함께 10월 부터 (김정식-2cca8b6e)님 합류.
- 현재 내부 인력의 대부분은 상품 소싱 및 배송에 투입되어, 사용자 특징이나 앱 서비스 개선점 등에 대해서는 정확히 파악하지 못하고 있음.
- 최근에는 매출을 올리기 위해 매장을 늘리는게 좋을지, 기존 매장관리에 주력하는게 좋을지 고민하고 있으나 뚜렷한 답을 찾지 못한 상황.



---



# **[결론 및 제언]**

- 리포트의 서두에는 분석을 통해 확인된 지표 현황 또는 인사이트에 대해 요약한 내용을 작성합니다.
- 순서상 결론 부분은 리포트의 말미에 작성되는게 맞으나 두괄식 형태로 결론에 대해 먼저 전달하고 상세 내용을 나열하는 형태로 문서를 작성하면 리포트 열람자의 시간을 아껴줄 수 있습니다.
- 만약 분석을 통해 도출된 가설이나, 제안하고 싶은 내용이 있다면 해당 내용을 포함해서 작성합니다. (중요 ✨)


---



# **[ 오프라인 매장 현황 ]**

## < 매장 정보 >

In [ ]:
#@title Make DataFrame

## << shopsInfosDf: 매장 정보 테이블 생성 >>

query_job = client.query("""
    with shopinfos as (
      select case when a.pickup_shop_name is null then '픽업존(통합)' else a.pickup_shop_name end as pickup_shop_name
           , case when a.pickup_shop_name is not null then a.region_1depth_name else '서울' end as region_1depth_name  
           , case when a.pickup_shop_name is not null then a.region_2depth_name else '-' end as region_2depth_name  
           , case when a.pickup_shop_name is not null then a.region_3depth_name else '-' end as region_3depth_name  
           , case when a.pickup_shop_name is null or a.pickup_shop_name = '픽업존(유승상가)' then '2021-07-22' 
                  else min(order_date) 
             end as shop_open_date -- (사업 담당자에게 확인한 정보를 기준으로) 최초 오픈일 수정
           -- 매장별 월별 매출
           , sum(case when order_year_month = '2022-05' then b.payment_amount end) as sales_05m
           , sum(case when order_year_month = '2022-06' then b.payment_amount end) as sales_06m
           , sum(case when order_year_month = '2022-07' then b.payment_amount end) as sales_07m
           , sum(case when order_year_month = '2022-08' then b.payment_amount end) as sales_08m
           , sum(case when order_year_month = '2022-09' then b.payment_amount end) as sales_09m
           , sum(case when order_year_month = '2022-10' then b.payment_amount end) as sales_10m
           , sum(case when order_year_month = '2022-11' then b.payment_amount end) as sales_11m
      from (
        -- 매장 정보 테이블
        select pickup_shop_name
             , region_1depth_name, region_2depth_name, region_3depth_name
        from `crested-drive-372522.localparm_dw.pickup_shop`
      ) as a
      inner join (
        -- 매장별 결제 테이블
        select order_date, order_year_month
             , payment_amount, pickup_shop_name
        from `crested-drive-372522.my_temp.daily_orders`
        where payment_status= 'completed' -- 결제 완료건만 집계
      ) as b
      on a.pickup_shop_name = b.pickup_shop_name
      group by rollup(a.pickup_shop_name, a.region_1depth_name, a.region_2depth_name, a.region_3depth_name)
    )

    select * 
    from shopinfos
    where region_3depth_name is not null
    order by shop_open_date, sales_11m desc
""")
shopsInfosDf = query_job.to_dataframe()

In [ ]:
#@title Show Table

## 월별 매출에 대해 <NA> 값을 0으로 변경 - DataFrame에 style.format 지정시 type error를 피하기 위한 처리 
shopsInfosDf.replace(np.NaN, 0, inplace=True)

## DataFrame의 스타일 형태를 수정해서, 각 월별 매출 크기에 따른 시각화 적용(주석을 순차적으로 제거후 실행하면서, 차이점을 확인해 보세요!)
shopsInfosDf.style.format({
    "sales_05m": "{:,.0f}",
    "sales_06m": "{:,.0f}",
    "sales_07m": "{:,.0f}",
    "sales_08m": "{:,.0f}",
    "sales_09m": "{:,.0f}",
    "sales_10m": "{:,.0f}",
    "sales_11m": "{:,.0f}",})#.bar(['sales_05m', 'sales_06m', 'sales_07m', 'sales_08m', 'sales_09m', 'sales_10m', 'sales_11m'], color='#d65f5f',)#.hide_index()

,pickup_shop_name,region_1depth_name,region_2depth_name,region_3depth_name,shop_open_date,sales_05m,sales_06m,sales_07m,sales_08m,sales_09m,sales_10m,sales_11m
0,픽업존(통합),서울,-,-,2021-07-22,"48,071,800","44,498,200","58,141,800","60,622,400","48,185,900","53,853,800","140,448,700"
1,픽업존(유승상가),서울,양천구,목동,2021-07-22,"48,071,800","33,845,600","31,887,400","32,043,300","23,377,900","26,862,500","47,837,300"
2,픽업존(메이비카페),서울,구로구,구로동,2022-06-14,0,"4,434,400","10,403,400","9,554,500","9,778,900","10,555,900","24,926,900"
3,픽업존(서울드림신용협동조합),서울,구로구,신도림동,2022-06-14,0,"1,569,700","5,038,900","6,875,000","5,606,400","6,414,300","8,840,100"
4,픽업존(GS25 목동3동점),서울,양천구,목동,2022-06-14,0,"1,834,700","4,893,500","6,012,200","4,817,000","5,401,800","8,242,500"
5,픽업존(참맛부대찌개아구찜),서울,영등포구,문래동3가,2022-06-15,0,"2,813,800","5,918,600","6,137,400","4,605,700","4,619,300","5,700,700"
6,픽업존(GS영등포당산점),서울,영등포구,당산동5가,2022-11-07,0,0,0,0,0,0,"32,906,900"
7,픽업존(다이소 난곡사거리점),서울,관악구,신림동,2022-11-07,0,0,0,0,0,0,"10,971,300"
8,픽업존(더현대서울),서울,영등포구,여의도동,2022-11-08,0,0,0,0,0,0,"1,023,000"


> **Note**

- 11월에 오픈한 (GS영등포당산점, 다이소 난곡사거리점)은 기존 다판다마켓의 판매지역이며, (더현대서울) 지점은 서울시 창업지원을 받아 입점 

> **Comment**

- 전체 매장의 11월 매출은 전월比 160% 상승
- 유승상가 지점의 경우 5월 이후 지속적인 매출 하락세였으나 11월에 전월比 매출 78% 상승
- 6월에 신규 오픈한 4개 지점 또한 8월 이후 매출 하락세에 들어갔으나 11월에 전월比 매출 (23% ~ 136%) 상승
- 11월에 신규 입점한 2개(GS영등포당산점, 다이소 난곡사거리점) 지점의 경우 11월 전체 매출의 약 30% 차지

> **Tip**

- 표나 그래프를 공유할 경우 특이점이나, 추가적으로 살펴봐야 하는 내용들 위주로 코멘트를 달아주며
- 분석가는 해당 지점들을 토대로 분석 포인트들을 발굴해 냅니다. (데이터 해석 과정을 통한 EDA)

## < 오프라인 매장 및 결제유저 분포 >

In [ ]:
#@title Make DataFrame

## << 11월 데이터 기준 - 매장별 결제 유저별 거주지 및 11월 누적 결제액  >>
query_job = client.query("""
    select a.user_id, b.x, b.y, a.pickup_shop_name, sum(payment_amount) as payment_amount
    from (
      -- 11월 결제 데이터
      select *
      from `crested-drive-372522.my_temp.daily_orders`
      where order_year_month = '2022-11'
        and payment_status = 'completed'
    ) as a
    inner join (
      -- 사용자 가입 정보(거주지)
      select *
      from `crested-drive-372522.my_temp.all_myinfos`
    ) as b
    on a.user_id = b.user_id
    group by a.user_id, b.x, b.y, a.pickup_shop_name
""")
novPickupUniquePu = query_job.to_dataframe()


## << 오프라인 매장 위치 >>
storesRegionDf = pd.DataFrame({
    'pickup_shop_name':['픽업존(메이비카페)', '픽업존(다이소 난곡사거리점)', '픽업존(서울드림신용협동조합)', '픽업존(참맛부대찌개아구찜)', '픽업존(GS영등포당산점)', '픽업존(유승상가)', '픽업존(GS25 목동3동점)', '픽업존(더현대서울)'],
    'region_1depth_name':['서울', '서울', '서울', '서울', '서울', '서울', '서울', '서울'],
    'region_2depth_name':['구로구', '관악구', '구로구', '영등포구', '영등포구', '양천구', '양천구', '영등포구'],
    'region_3depth_name':['구로동', '신림동', '신도림동', '문래동3가', '당산동5가', '목동', '목동', '여의도동'],
    'h_code':[1153054000, 1162076500, 1153051000, 1156060500, 1156056000, 1147055000, 1147053000, 1156054000],
    'x':[37.4849288, 37.48145951, 37.5095675, 37.51627575, 37.53237867, 37.5398954, 37.54638929, 37.52519132],
    'y':[126.9002594, 126.9142068, 126.8820361, 126.8973904, 126.8968009, 126.881942, 126.8644345, 126.9291128],
    
})

In [ ]:
#@title Visualization

## Folium Document > https://python-visualization.github.io/folium/  <- Quick Start 부분을 참고해 주세요 😉 
from folium.plugins import MarkerCluster 

## << 핀 포인트 - 초기 지도 위치 설정 >>
latitude = 37.5161 # 위도
longitude = 126.8967 # 경도

m = folium.Map(
    location=[latitude, longitude],
    zoom_start=13,
)

mrkerCluster = MarkerCluster().add_to(m)  

## << 매장 위치 표시 >>
for lat, long, name in zip(storesRegionDf['x'], storesRegionDf['y'], storesRegionDf['pickup_shop_name']):
    folium.Marker(location = [lat, long], popup=name, icon=folium.Icon(color='red', icon='star')).add_to(m)

## << 구매 유저 위치 표시 >>
for lat, long in zip(novPickupUniquePu['x'], 
                     novPickupUniquePu['y']):
    folium.Marker([lat, long], icon = folium.Icon(color="blue")).add_to(mrkerCluster)
    
m

In [ ]:
## << [참고] Python zip() 함수 >>
#  ㄴ 여러 개의 순회 가능한 객체를 인자로 받아서, 각 객체가 담고 있는 원소를 튜플 형태로 접근할 수 있는 반복자를 반환 >>

for a in zip(storesRegionDf['x'], storesRegionDf['y'], storesRegionDf['pickup_shop_name']):
  print(a);
  print([lat, long]);

(37.4849288, 126.9002594, '픽업존(메이비카페)')
[37.5093, 126.8847]
(37.48145951, 126.9142068, '픽업존(다이소 난곡사거리점)')
[37.5093, 126.8847]
(37.5095675, 126.8820361, '픽업존(서울드림신용협동조합)')
[37.5093, 126.8847]
(37.51627575, 126.8973904, '픽업존(참맛부대찌개아구찜)')
[37.5093, 126.8847]
(37.53237867, 126.8968009, '픽업존(GS영등포당산점)')
[37.5093, 126.8847]
(37.5398954, 126.881942, '픽업존(유승상가)')
[37.5093, 126.8847]
(37.54638929, 126.8644345, '픽업존(GS25 목동3동점)')
[37.5093, 126.8847]
(37.52519132, 126.9291128, '픽업존(더현대서울)')
[37.5093, 126.8847]


> **Comment**
- 주거 밀집 지역에 위치한 매장일수록 결제 유저가 몰려있는 것을 확인할 수 있으며
- 유사한 입지 임에도, 매장별로 결제유저수에 차이가 나는 것으로 보아 매장별 퍼포먼스(구매력/재구매율/선호상품/거리 분포 등) 분석이 필요

> **Tip**
- 오프라인 기반 서비스의 경우 외부요인(코로나, 입지, 경쟁사 입점, 매장 임대료 등)에 의한 서비스 의존도가 크기 때문에 
- 객관적인 관점으로 데이터를 바라보기 위해서는 실제 매장 위치와 주변 입지에 대한 이해가 필요하며
- 필요시 현장답사를 하는 것도 데이터를 이해하는데 큰 도움이 됩니다.  

# **[ 사용자 유입 현황 분석 ]**

## < 요약 ✨ >


> **다판다마켓 인수 및 신규매장 오픈 효과**
  - 11월 MAU는 역대 최고 수준인 6,352명 달성 (전월比 55% 증가)
  - 다판다마켓의 기존 판매지역(GS영등포당산점, 다이소 난곡사거리점) 외에도 기존 로컬팜 판매지역의 AU 상승이 관찰될 것으로 보아 
  - 다판다마켓 인수가 어떤식으로 AU 증가에 영향을 미쳤는지 추가 확인 필요

> **기존 유저들의 이탈 속도 증가**
  - 22년 가입 유저군의 경우 사용자의 신규 유입보다 이탈 속도가 빨라 8월 ~ 10월 동안 MAU 하락세에 있었으며
  - 특히 유저들의 모객 시점이 22년 하반기로 갈수록 이탈속도가 빨라지고 있음
  - 이러한 이탈 가속 현상은 다수의 지역에서 공통적으로 발생하고 있으므로 추후 별도의 이탈 원인 분석 필요


## < MAU, NRU >

In [ ]:
#@title Make DataFrame

## << mauDf: 월간 활성 유저수 >>
query_job = client.query("""
    select login_year_month
         , count(distinct user_id) as login_user_cnt
    from `crested-drive-372522.my_temp.daily_login_users`
    group by login_year_month
    order by login_year_month    
""")
mauDf = query_job.to_dataframe()

## << monthlyNruDf: 월간 가입 유저수 >>
query_job = client.query("""
    select create_year_month, count(distinct user_id) as user_cnt
    from `crested-drive-372522.my_temp.all_myinfos`
    group by create_year_month
    order by create_year_month
""")
monthlyNruDf = query_job.to_dataframe()

In [ ]:
#@title Visualization
import plotly.graph_objects as go

## Step.1 - 강조하고 싶은 부분만 crimson 색으로 표시
#  색상 참고: https://www.devkuma.com/docs/html/html-%EC%83%89-color/
colors = ['lightgray',] * len(mauDf)
colors[len(mauDf) - 1] = 'crimson'

## Step.2 - fig안에 go.Bar(), go.Scatter() 그래프 담기
fig = go.Figure()
fig.add_trace(
    go.Bar(    
        x=mauDf['login_year_month'], 
        y=mauDf['login_user_cnt'],
        name='MAU(월간 활성 유저수)',
        text=mauDf['login_user_cnt'], 
        textposition='auto', 
        texttemplate='%{text:,}명',
        marker_color=colors # MAU 11월 색 강조
    )
)

fig.add_trace(
    go.Scatter(
        x=monthlyNruDf[monthlyNruDf['create_year_month'] >= '2022-01']['create_year_month'],
        y=monthlyNruDf[monthlyNruDf['create_year_month'] >= '2022-01']['user_cnt'],
        name='NRU(신규 유저수)', 
        mode='lines+markers+text', 
        text=monthlyNruDf[monthlyNruDf['create_year_month'] >= '2022-01']['user_cnt'],  
        textposition='top center',
        texttemplate='%{text:,}명',
    )
)

## Step.2 - fig의 layout 설정 변경
fig.update_layout(
    {
        "title": {
            "text": "<b>< 월간 MAU, NRU ><b>",
            "x": 0.5,
            "y": 0.9,
            "font": {
                "size": 13
            }
        },
        "yaxis": {
            "title": "MAU, NRU(명)", "tickformat": ','
        },
        "xaxis": {
            "tick0": "2022-01",
            "dtick": 'M1',
            "tickformat": "%Y-%m"
        },
        "template":'plotly_white',
        "legend": {'orientation': 'h', 'x': 0, 'y': 1.2}
    },
)

## [참고] fig의 layout 속성중 일부를, 개별적으로 설정해 주는 방법도 있습니다. 
#  ㄴ dict()함수는 https://wikidocs.net/16 참고!
# fig.update_layout(legend = dict(orientation = "h",
#                                 x=0,
#                                 y=1.2,
# ))
#

## Step.3 - fig의 y축을 기준으로, 직사각형 영역 표시 또는 텍스트를 남깁니다.
fig.add_vrect(x0="2021-12-15", 
              x1="2022-04-15",
              annotation_text="* 활성 유저수 지표는 22년 5월부터 확인 가능", 
              annotation_position="top left",
              fillcolor="black", opacity=0.1, line_width=0)

fig.add_vrect(x0="2022-07-18", 
              x1="2022-10-15",
              annotation_text="* MAU 점진적 하락 구간", 
              annotation_position="top left",
              fillcolor="hotpink", opacity=0.1, line_width=0)


## Step.4 - 그래프 출력 
fig.show(renderer='colab')

> **Comment**

- 7월 이후 MAU 볼륨이 점진적 하락 추세에 들어섰으나 11월 다판다마켓 인수 이후 6,352명 달성 (전월比 55% 증가) 
- 11월 NRU는 2,907명으로 전월比 258%(2,096명) 증가

In [ ]:
## fig의 layout 정보 살펴보기
fig.layout

## < 지역별 MAU, NRU >

In [ ]:
#@title Make DataFrame

## << novShopAndPuRegion: 11월 매출 기준 매장별 구매자 거주지역별 매출 >>
query_job = client.query("""

-- 지역별 구매유저 결제정보
with pu_region as (
  select a.user_id, a.order_year_month, a.order_date, a.payment_amount, a.pickup_shop_name, b.region_3depth_name as region
  from `my_temp.daily_orders` as a
  left join `my_temp.all_myinfos` as b
  on a.user_id = b.user_id
),

-- 11월 매장별, 지역별, 월별 누적 매출
nov_shop_and_region as (
  select order_year_month, pickup_shop_name
       -- pivot error 방지를 위한 region값 영문 변환 처리
       , case when region = '신림동' then 'Sillim_dong' 
              when region = '조원동' then 'Jowon_dong'
              when region = '구로2동' then 'Guro2_dong'
              when region = '신도림동' then 'Sindorim_dong'
              when region = '독산3동' then 'Doksan3_dong'
              when region = '목동' then 'Mok_dong'
              when region = '당산동4가' then 'Dangsandong4_ga'
              when region = '당산동5가' then 'Dangsandong5_ga'
              when region = '대림2동' then 'Daerim2_dong'
              when region = '문래동3가' then 'Mullaedong3_ga'
              when region = '양평동3가' then 'Yangpyeongdong3_ga'
              when region = '양평동4가' then 'Yangpyeongdong4_ga'
              when region = '여의도동' then 'Yeouido_dong'
         else 'none'
         end as region
       , sum(payment_amount) as sales
  from pu_region
  group by order_year_month, pickup_shop_name, region
  having order_year_month = '2022-11'
  order by pickup_shop_name, region
)

select *
from nov_shop_and_region
pivot(min(sales) for region in ('Sillim_dong', 'Jowon_dong', 'Guro2_dong', 'Sindorim_dong', 'Doksan3_dong', 'Mok_dong', 'Dangsandong4_ga', 'Dangsandong5_ga', 'Daerim2_dong', 'Mullaedong3_ga', 'Yangpyeongdong3_ga', 'Yangpyeongdong4_ga', 'Yeouido_dong'))
order by case when pickup_shop_name = '픽업존(유승상가)' then 1
              when pickup_shop_name = '픽업존(메이비카페)' then 2 
              when pickup_shop_name = '픽업존(서울드림신용협동조합)' then 3 
              when pickup_shop_name = '픽업존(GS25 목동3동점)' then 4 
              when pickup_shop_name = '픽업존(참맛부대찌개아구찜)' then 5 
              when pickup_shop_name = '픽업존(GS영등포당산점)' then 6 
              when pickup_shop_name = '픽업존(다이소 난곡사거리점)' then 7 
              when pickup_shop_name = '픽업존(더현대서울)' then 8 
          end
""")
novShopAndPuRegion = query_job.to_dataframe()

In [ ]:
#@title < 매장별 거주 지역별 11월 매출 테이블 >

novShopAndPuRegion.fillna(0) # novShopAndPuRegion 데이터프레임에서 <NA> 값을 0으로 변환

,order_year_month,pickup_shop_name,Sillim_dong,Jowon_dong,Guro2_dong,Sindorim_dong,Doksan3_dong,Mok_dong,Dangsandong4_ga,Dangsandong5_ga,Daerim2_dong,Mullaedong3_ga,Yangpyeongdong3_ga,Yangpyeongdong4_ga,Yeouido_dong
0,2022-11,픽업존(유승상가),0,0,0,0,0,51315800,0,0,0,2748000,0,0,0
1,2022-11,픽업존(메이비카페),0,3092100,2149800,0,837700,9565500,0,0,11978700,325300,0,0,0
2,2022-11,픽업존(서울드림신용협동조합),0,0,0,9418500,0,0,0,0,0,76100,0,0,0
3,2022-11,픽업존(GS25 목동3동점),0,0,0,179400,0,8660200,0,0,0,0,0,0,0
4,2022-11,픽업존(참맛부대찌개아구찜),0,0,0,0,0,0,0,0,0,6137600,0,0,0
5,2022-11,픽업존(GS영등포당산점),0,0,0,0,72000,813400,10493300,21598900,0,0,2232800,1193600,173700
6,2022-11,픽업존(다이소 난곡사거리점),10452400,0,0,0,1477100,0,0,0,88600,0,0,0,84900
7,2022-11,픽업존(더현대서울),0,0,0,0,0,0,0,0,0,0,0,0,1199500


> **Comment**

- 지역별 MAU를 확인하기에 앞서, 매장별 실 수요지역을 파악하기 위해 매장별 거주지역별 11월 매출 테이블 집계

In [ ]:
#@title Make DataFrame - 1

## << mauByRegionDf: 월간 지역별 활성 유저수 >>
query_job = client.query("""
  
  -- 일간 접속유저 테이블(+ 거주지역) 
  with active_user as (
    select a.login_date, a.login_year_month, a.user_id, b.region_1depth_name, b.region_2depth_name, region_3depth_name
    from `crested-drive-372522.my_temp.daily_login_users` as a
    inner join `crested-drive-372522.my_temp.all_myinfos` as b
    on a.user_id = b.user_id
  ),

  -- 지역별 월간 접속유저 
  montly_active_user as (
     select region, login_year_month, user_cnt
         , lag(user_cnt, 1) over(partition by region order by login_year_month) as bef_1m_user_cnt
         , rank() over(partition by region order by login_year_month) as rank_asc -- 지역별 최초 접속 시점을 가려내기 위함
     from (
          -- 지역(region)별, 월별 MAU 집계
          select concat(region_2depth_name, '_', region_3depth_name) as region
               , login_year_month
               , count(distinct user_id) as user_cnt
          from active_user
          group by region, login_year_month
     ) as a
     order by region, login_year_month
  )

  select region, login_year_month
       , rank_asc
       , user_cnt
       , bef_1m_user_cnt
       , (user_cnt - bef_1m_user_cnt)/bef_1m_user_cnt as rate_of_change
       -- 각 지역별 11월 AU의 전월대비 증감률을 계산하고, 이를 통해 유저 그룹을 나눈다.
       , last_value((user_cnt - bef_1m_user_cnt)/bef_1m_user_cnt) over(partition by region order by login_year_month rows between unbounded preceding and unbounded following) as nov_rate_of_change
       , case when rank_asc = 1 and login_year_month = '2022-11' then 'new'
              when last_value((user_cnt - bef_1m_user_cnt)/bef_1m_user_cnt) over(partition by region order by login_year_month rows between unbounded preceding and unbounded following) < 0 then 'decrease'
              when last_value((user_cnt - bef_1m_user_cnt)/bef_1m_user_cnt) over(partition by region order by login_year_month rows between unbounded preceding and unbounded following) between 0 and 0.1 then 'increase_0_10'
              when last_value((user_cnt - bef_1m_user_cnt)/bef_1m_user_cnt) over(partition by region order by login_year_month rows between unbounded preceding and unbounded following) <= 0.2 then 'increase_10_20'
              when last_value((user_cnt - bef_1m_user_cnt)/bef_1m_user_cnt) over(partition by region order by login_year_month rows between unbounded preceding and unbounded following) <= 0.3 then 'increase_20_30'
              else 'increase_30_more'
         end as user_group
  from montly_active_user
  order by region, login_year_month
""")
mauByRegionDf = query_job.to_dataframe()

In [ ]:
#@title Visualization-1

## << 지역별 MAU - Bar Chart >>

import plotly.express as px

## Step.1 - fig에 px.bar() 그래프 담기
fig = px.bar(mauByRegionDf, 
              x="login_year_month", 
              y="user_cnt", 
              color="region", 
              text="user_cnt",
              # facet_row="region_name",
             )

## Step.2 - text 표현 형식과, 출력 위치를 조정
fig.update_traces(texttemplate='%{text:,}명', textposition='inside') 

## Step.3 - fig의 layout 설정 변경
fig.update_layout(
    {
        "title": {
            "text": "<b>< 월간 지역별 MAU ><b>",
            "x": 0.5,
            "y": 0.95,
            "font": {
                "size": 13
            }
        },
        "yaxis": { "title": "MAU", "tickformat": ','  }, 
        "xaxis": {
            "title": "",
            "tick0": "2022-05",
            "dtick": 'M1',
            "tickformat": "%Y-%m"
        },
        "legend_title_text": "거주지역",
        "template":'plotly_white',
    }
)

## Step.4 - fig의 y축을 기준으로, 직사각형 영역 표시 또는 텍스트를 남깁니다.
fig.add_vrect(x0="2022-10-15", 
              x1="2022-11-17",
              annotation_text="* 11월 신규매장 오픈 이후, 기존 지역의 AU 볼륨 전월比 상승 (문래동3가 제외)", 
              annotation_position="top right",
              fillcolor="skyblue", opacity=0.25, line_width=0)

## Step.5 - 그래프 출력
fig.show()

> **Comment**

- 11월 신규 매장(GS영등포당산점, 다이소 난곡사거리점, 더현대서울)	오픈에 따라 여의도동, 양평동4가 지역주민이 신규로 유입되었으며
- 10월 기준 AU 볼륨이 500명 이하였던 지역들 위주로 11월 AU 30% 이상 증가 추세   
- 지역별 AU 비교는 아래의 < [별첨] 월간 지역별 MAU, NRU 추이 비교 > 장표 참고

> **Tip**

- 장표에 코멘트를 다는 경우 추세나 변화를 파악할 수 있는 요약 통계량을 활용합니다.
- 예를들어 '크게 늘었다' 라는 표현보다는 'n% 상승'이라고 표현하는게 정보를 좀 더 구체적으로 전달 할 수 있습니다.

In [ ]:
#@title Make DataFrame-2

## << monthlyNruByRegionDf: 월간 지역별 가입 유저수 >>
query_job = client.query("""

  -- 신규 가입 유저 정보 테이블(+ region 컬럼 추가)
  with registere_user as (
    select user_id, create_date, create_year_month
         , region_1depth_name, region_2depth_name, region_3depth_name
         , concat(region_2depth_name, '_', region_3depth_name) as region
    from `crested-drive-372522.my_temp.all_myinfos`
  ),

  -- 지역(region)별, 월별 가입 유저수 테이블
  montly_registere_user as (
     select region, create_year_month, user_cnt
         , lag(user_cnt, 1) over(partition by region order by create_year_month) as bef_1m_user_cnt
         , rank() over(partition by region order by create_year_month) as rank_asc -- 지역별 최초 가입 시점을 가려내기 위함
     from (
          select region
               , create_year_month
               , count(distinct user_id) as user_cnt
          from registere_user
          group by region, create_year_month
          having create_year_month >= '2022-05'
     ) as a
     order by region, create_year_month
  )

  select region, create_year_month
       , rank_asc
       , user_cnt
       , bef_1m_user_cnt
       -- 각 지역별 11월 NRU의 전월대비 증감률을 계산하고, 이를 통해 유저 그룹을 나눈다.
       , (user_cnt - bef_1m_user_cnt)/bef_1m_user_cnt as rate_of_change
       , last_value((user_cnt - bef_1m_user_cnt)/bef_1m_user_cnt) over(partition by region order by create_year_month rows between unbounded preceding and unbounded following) as nov_rate_of_change
       , case when rank_asc = 1 and create_year_month = '2022-11' then 'new'
              when last_value((user_cnt - bef_1m_user_cnt)/bef_1m_user_cnt) over(partition by region order by create_year_month rows between unbounded preceding and unbounded following) < 0 then 'decrease'
              when last_value((user_cnt - bef_1m_user_cnt)/bef_1m_user_cnt) over(partition by region order by create_year_month rows between unbounded preceding and unbounded following) between 0 and 0.1 then 'increase_0_10'
              when last_value((user_cnt - bef_1m_user_cnt)/bef_1m_user_cnt) over(partition by region order by create_year_month rows between unbounded preceding and unbounded following) <= 0.2 then 'increase_10_20'
              when last_value((user_cnt - bef_1m_user_cnt)/bef_1m_user_cnt) over(partition by region order by create_year_month rows between unbounded preceding and unbounded following) <= 0.3 then 'increase_20_30'
              else 'increase_30_more'
         end as user_group
  from montly_registere_user
  order by region, create_year_month
""")
monthlyNruByRegionDf = query_job.to_dataframe()



##---< 시각화 편의를 위한 데이터프레임 병합 & 컬럼명 변경 & 값 변경 >-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


## < monthlyAuAndNru: 월간 지역별 활성 유저수, 가입 유저수 >

mauByRegionDf = mauByRegionDf.rename(columns={'login_year_month':'year_month'})
mauByRegionDf['category'] = 'AU'
monthlyNruByRegionDf = monthlyNruByRegionDf.rename(columns={'create_year_month':'year_month'})
monthlyNruByRegionDf['category'] = 'NRU'
monthlyAuAndNru = pd.concat([mauByRegionDf, monthlyNruByRegionDf], ignore_index=True) 

# user_group 한글로 변경
monthlyAuAndNru['user_group'] = np.where(monthlyAuAndNru['user_group'] == 'new', '신규지역', monthlyAuAndNru['user_group'])
monthlyAuAndNru['user_group'] = np.where(monthlyAuAndNru['user_group'] == 'decrease', '감소지역', monthlyAuAndNru['user_group'])
monthlyAuAndNru['user_group'] = np.where(monthlyAuAndNru['user_group'] == 'increase_0_10', '10%이하_증가지역', monthlyAuAndNru['user_group'])
monthlyAuAndNru['user_group'] = np.where(monthlyAuAndNru['user_group'] == 'increase_10_20', '10%초과_20%이하_증가지역', monthlyAuAndNru['user_group'])
monthlyAuAndNru['user_group'] = np.where(monthlyAuAndNru['user_group'] == 'increase_20_30', '20%초과_30%이하_증가지역', monthlyAuAndNru['user_group'])
monthlyAuAndNru['user_group'] = np.where(monthlyAuAndNru['user_group'] == 'increase_30_more', '30%_초과_증가지역', monthlyAuAndNru['user_group'])


In [ ]:
#@title Visualization-2

## << 지역별 AU, NRU >>

import plotly.express as px

## Step.1 - fig에 px.line() 그래프 담기
fig = px.line(monthlyAuAndNru, 
              x="year_month", 
              y= "user_cnt", 
              color="region",
              facet_col = "user_group",
              facet_row = "category",
              # facet_row_spacing = 0.05, #  (float between 0 and 1) – Spacing between facet rows, in paper units. Default is 0.03 or 0.07 when facet_col_wrap is used.
              markers=True,
              category_orders={"user_group": ["신규지역", "감소지역", "10%이하_증가지역", "30%_초과_증가지역"]}, # '10%초과_20%이하_증가지역', '20%초과_30%이하_증가지역' 은 데이터가 없어서 제외
              hover_data={"category":False, "user_group":False}
              )

## Step.2 - fig의 layout 설정 변경
fig.update_layout(
    {
        "title": {
            "text": "<b>< [별첨] 월간 지역별 MAU, NRU 추이 비교 ><b>",
            "x": 0.5,
            "y": 0.99,
            "font": {
                "size": 15
            }
        },
        "yaxis1": {
            "title": "NRU(명)",
            "tickformat": ","
        },
        "yaxis5": {
            "title": "MAU(명)",
            "tickformat": ","
        },
        "legend_traceorder": "normal", #normal(default - 데이터 순서대로), reversed(역순)
        "legend_title_text": "거주 지역"
    },
    height = 550 
)

## Step.3 - fig의 x축 상세 설정
fig.update_xaxes(showticklabels=True) # 1행, 2행 모두 x축 thick 출력
fig.update_xaxes(tickformat = '%m월', dtick = "M1", tick0 = "2022-05", title = "")

## Step.4 - 그래프 마우스 오버시 hovermode x축으로 설정
fig.update_layout(hovermode="x")

## Step.5 - 그래프 출력
fig.show()

> **Note**

- user_group 분류기준: 11월 지표의 전월比 증감률을 기준으로 계산

> **Comment**

- 11월 MAU의 경우 문래동3가 지역을 제외한 모든 지역이 전월比 상승
  - 10% 이하 증가 지역: 목동(2559명), 신도림동(352명)
  - 30% 초과 증가 지역: 대림2동(669명), 당산동5가(651명), 당산동4가(441명), 신림동(386명), 조원동(287명), 구로2동(281명), 독산3동(101명), 양평동3가(37명)  
  - 문래동3가의 MAU는 518명으로 전월比 -5.5% 감소
- 목동과 신도림동의 11월 NRU는 전월比 각각 -12.5%, -40.3% 감소했으나 MAU는 각각 1.2%, 0.3% 수준 소폭 상승

> **Tip**

- 앞서 전달한 Bar chart에서 그룹간 상대비교가 어려운 경우, Line chart로 재표현 해서 추가 제공하기도 합니다.

## < 가입 연도별/월별 MAU >



In [ ]:
#@title Make DataFrame-1

## << yearlyAuDf: 가입 연도별 활성 유저수 >>
query_job = client.query("""
  with active_user as (
    select a.login_date, a.login_year_month, b.create_date, format_date("%Y", b.create_date) as create_year, a.user_id, b.region_1depth_name, b.region_2depth_name, region_3depth_name
    from `crested-drive-372522.my_temp.daily_login_users` as a
    inner join `crested-drive-372522.my_temp.all_myinfos` as b
    on a.user_id = b.user_id
  )

  select login_year_month, create_year 
       , count(distinct user_id) as user_cnt
  from active_user
  group by login_year_month, create_year
  order by login_year_month, create_year
""")
yearlyAuDf = query_job.to_dataframe()

In [ ]:
#@title Visualization-1

## << 가입 연도별 월간 활성 유저수 - Bar Chart >>

import plotly.express as px

## Step.1 - fig에 px.bar() 그래프 담기
fig = px.bar(yearlyAuDf, 
              x="login_year_month", 
              y="user_cnt", 
              color="create_year", 
              text="user_cnt",
             )

## Step.2 - text 표현 형식과, 출력 위치를 조정
fig.update_traces(texttemplate='%{text:,}명', textposition='inside') 

## Step.3 - fig의 layout 설정 변경
fig.update_layout(
    {
        "title": {
            "text": "<b>< 가입 연도별 MAU ><b>",
            "x": 0.5,
            "y": 0.95,
            "font": {
                "size": 13
            }
        },
        "yaxis": { "title": "MAU", "tickformat": ','  }, 
        "xaxis": {
            "title": "",
            "tick0": "2022-05",
            "dtick": 'M1',
            "tickformat": "%Y-%m"
        },
        "legend_title_text": "가입 연도",
        "template":'plotly_white',
    }
)

## Step.4 - fig의 y축을 기준으로, 직사각형 영역 표시 또는 텍스트를 남깁니다.
fig.add_vrect(x0="2022-07-17", 
              x1="2022-10-16",
              annotation_text="* 2022년 가입유저 기준, 유입보다 이탈 속도가 빠른 구간", 
              annotation_position="top left",
              # annotation_font_size=14,
              fillcolor="hotpink", opacity=0.25, line_width=0)

fig.add_vrect(x0="2022-10-18", 
              x1="2022-11-15",
              annotation_text="* 신규 매장 오픈", 
              annotation_position="top left",
              # annotation_font_size=14,
              fillcolor="skyblue", opacity=0.25, line_width=0)

## Step.5 - 그래프 출력
fig.show()

> **Comment**

- 2022년 가입 그룹의 MAU는 7월 이후 10월까지 하락세로 전환됨 (매달 500명 이상의 신규가입자가 발생하고 있으나 유저의 이탈 속도가 더 빠름)
- 2022년 가입 그룹의 11월 MAU는 신규매장 오픈 & 다판다마켓 인수에 따라 역대 최고 수준(5,042명) 달성

> **Tip**

- 지표를 (가입시점/지역/결제유무/커스텀그룹) 등으로 나누어 보고자 할 경우 한번에 여러 Depth로 나누어 보기 보다는, 가장 낮은 수준의 Depth 부터 순차적으로 나누어 관찰하는 방법을 추천해 드립니다.
- 지표를 나누는 Depth가 깊고 복잡할수록 모수가 줄어들어 편향된 해석을 할 수 있습니다.


In [ ]:
#@title Make DataFrame-2

## << monthlyAuDf: 가입 연월별 활성 유저수 >>
query_job = client.query("""
  with active_user as (
    select a.login_date, a.login_year_month, b.create_date, create_year_month, a.user_id, b.region_1depth_name, b.region_2depth_name, region_3depth_name
    from `crested-drive-372522.my_temp.daily_login_users` as a
    inner join `crested-drive-372522.my_temp.all_myinfos` as b
    on a.user_id = b.user_id
  )

  select login_year_month, create_year_month 
       , count(distinct user_id) as user_cnt
  from active_user
  group by login_year_month, create_year_month
  order by login_year_month, create_year_month
""")
monthlyAuDf = query_job.to_dataframe()

In [ ]:
#@title Visualization-2

## << 가입 연월별 월간 활성 유저수 - Bar Chart >>

import plotly.express as px

## Step.1 - fig에 px.bar() 그래프 담기
fig = px.bar(monthlyAuDf, 
              x="login_year_month", 
              y="user_cnt", 
              color="create_year_month", 
              text="user_cnt",
              # facet_row="region_name",
             )

## Step.2 - text 표현 형식과, 출력 위치를 조정
fig.update_traces(texttemplate='%{text:,}명', textposition='inside') 

## Step.3 - fig의 layout 설정 변경
fig.update_layout(
    {
        "title": {
            "text": "<b>< 가입 연월별 MAU ><b>",
            "x": 0.5,
            "y": 0.95,
            "font": {
                "size": 13
            }
        },
        "yaxis": { "title": "MAU", "tickformat": ','  }, 
        "xaxis": {
            "title": "",
            "tick0": "2022-05",
            "dtick": 'M1',
            "tickformat": "%Y-%m"
        },
        "legend_title_text": "가입 연월",
        "template":'plotly_white',
        "height": 500
    }
)

## Step.4 - 그래프 출력
fig.show()

> **Comment**

- **그룹별 MAU 증감 추이를 살펴봤을때 신규매장 오픈 이슈가 없었다면 11월 MAU는 10월 보다 낮거나 비슷한 수준이 유지되었을 것으로 추정됨**
  - 11월 예상 MAU: 3,946명 = 11월 신규 유입(811명) + 기존 유저들의 11월 MAU(300명 + 2,835명)
  - 가정1: 10월과 동일한 수준의 신규 유입 발생(811명)
  - 가정2: 10월 모객 그룹은 11월 실데이터 사용(300명)
  - 가정3: 9월 → 10월 MAU 증감률을 11월 MAU 계산에 단순 적용(2,835명)
- **11월 신규매장 오픈 및 다판다마켓 인수는 어떻게 기존 유저들의 MAU 감소를 방어에 영향을 주었는지 확인 필요**

In [ ]:
#@title Make DataFrame-3

## << auChurnedRateDf: 가입 연월별 - 최초 유입시점 볼륨 대비 미접속률 >>
query_job = client.query("""

  -- 일간 접속유저 테이블(+ 거주지역, 가입시점) 
  with active_user as (
    select a.login_date, a.login_year_month, b.create_date, b.create_year_month, a.user_id, b.region_1depth_name, b.region_2depth_name, region_3depth_name
    from `crested-drive-372522.my_temp.daily_login_users` as a
    inner join `crested-drive-372522.my_temp.all_myinfos` as b
    on a.user_id = b.user_id
  ),

  -- 가입월별, 접속월별 접속 유저수 & 가입시점 기준 최초 접속 유저수 테이블
  active_user_by_create_month as (
     select login_year_month, create_year_month, user_cnt
          , first_value(user_cnt) over(partition by create_year_month order by login_year_month) as first_user_cnt -- 가입 시점 기준 최초 접속 유저수
     from (
          select login_year_month, create_year_month 
               , count(distinct user_id) as user_cnt
          from active_user
          group by login_year_month, create_year_month
     ) as a
     order by create_year_month, login_year_month
  )

  select login_year_month, create_year_month
       , substr(create_year_month, 1, 4) as create_year
       , concat('M+', rank() over(partition by create_year_month order by login_year_month) - 1) as time_order
       , user_cnt
       , first_user_cnt
       , round(user_cnt/first_user_cnt, 4) as rate_of_change
  from active_user_by_create_month 
""")
auChurnedRateDf = query_job.to_dataframe()
auChurnedRateDf = auChurnedRateDf.rename(columns={"first_user_cnt":"M+0_user_count"})



##---< 시각화 편의를 위한 데이터프레임 형태 변경 >-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## << auChurnedCntAndRateDf: 가입 연월별 - 월간(접속 유저수, 재접속률) - M+0부터 추적 가능한 2022-05 유입 유저부터 잘라냄 >>
auChurnedCntAndRateDf = auChurnedRateDf[auChurnedRateDf['create_year_month'] >= '2022-05']\
.sort_values(by=["create_year_month", "login_year_month"])\
.melt(id_vars=["create_year_month", "login_year_month", "create_year", "time_order"], value_vars=["user_cnt", "rate_of_change"]) 

In [ ]:
#@title Visualization-3

## << 가입연월별 최초유입시점대비 재접속률 변화 >>
import plotly.express as px

## Step.1 - fig에 px.line() 그래프 담기
fig = px.line(auChurnedCntAndRateDf, 
              x="time_order", 
              y="value", 
              color="create_year_month",
              facet_col = "variable",
              facet_col_spacing = 0.05, #  (float between 0 and 1) – Spacing between facet rows, in paper units. Default is 0.03 or 0.0.7 when facet_col_wrap is used.
              markers=True,
              category_orders={"create_year_month": ["2021-07", "2021-08", "2021-09", "2021-10", "2021-11", "2021-12", 
                                                     "2022-01", "2022-02", "2022-03", "2022-04", "2022-05", "2022-06", "2022-07", "2022-08", "2022-09", "2022-10", "2022-11", "2022-12"]},
              hover_data = {"variable":False, "time_order": False} # 마우스 오버시 노출하고 싶지 않은 데이터 제외 처리
              )

## Step.2 - fig의 layout 설정 변경
fig.update_layout(
    {
        "title": {
            "text": "<b>< [별첨] 가입 연월별 AU의 M+n 재접속률 ><b>",
            "x": 0.5,
            "y": 0.99,
            "font": {
                "size": 13
            }
        },
        "xaxis": {
            "title": "",
            "tick0": "1",
        },
        "xaxis2": {
            "title": "",
            "tick0": "1",
        },
        "legend_traceorder": "normal", #normal(default - 데이터 순서대로), reversed(역순)
        "legend_title_text": "가입 연월"
    },
    height = 350 
)


## Step.3 - fig의 화살표 설정
fig.add_annotation(
  x="M+1",  # arrows' head
  y=0.4,  # arrows' head
  ax="M+1",  # arrows' tail
  ay=0.6,  # arrows' tail
  xref='x2',
  yref='y2',
  axref='x2',
  ayref='y2',
  text='점진적 하락세',  # if you want only the arrow: ''
  font=dict(size=14, color='black'), showarrow=True,
  arrowhead=3,
  arrowsize=1,
  arrowwidth=2,
  arrowcolor='hotpink',
  row=1, col=2
)

## Step.4 - fig의 y축을 기준으로, 직사각형 영역 표시 또는 텍스트를 남깁니다.
fig.add_vrect(x0="M+0", 
              x1="M+2",
              annotation_text="", 
              # annotation_position="top left",
              fillcolor="hotpink", opacity=0.15, line_width=0) # https://plotly.com/python/horizontal-vertical-shapes/#horizontal-and-vertical-lines-in-dash 

## Step.5 - fig의 y축 상세 설정
fig.update_yaxes(title='활성 유저수(명)', row=1, col=1)
fig.update_yaxes(tickformat = ',', row=1, col=1)
fig.update_yaxes(title='재접속률(%)', row=1, col=2)
fig.update_yaxes(tickformat = '.0%', row=1, col=2)
fig.update_yaxes(matches=None, showticklabels=True) # matches=None (y축을 독립적으로 설정)

## Step.6 - 그래프 마우스 오버시 hovermode x축으로 설정
fig.update_layout(hovermode="x")

## Step.7 - 그래프 출력
fig.show()

> **Comment**

- **22년 후반으로 갈수록 모객 집단의 이탈률이 가속화 되고 있는 것으로 추정**
  - **관련 근거 1)** 22년 5월부터 10월까지(8월 제외) 각 그룹별 M+1 재접속률이 지속적으로 하락
  - **관련 근거 2)** 22년 10월 보다 모수가 200명 이상 큰 7월, 6월의 M+1 재접속률은 10월比 약 11%p 높은 48% 수준 
- **이탈률 가속 현상이 특정 지역의 이슈인지 확인 필요**

> **Tip**

- 위와 같이 특정 시점의 수치를 100%로 두고 시간의 흐름에 따라 변동 수준을 확인하는 차트를 팬차트 라고 합니다.
- 증감률(%)을 비교하는 경우 모수의 크기도 함께 비교되어야 정확한 판단이 가능합니다.
- 모수가 클수록 허수가 섞여있을 가능성이 높고 그만큼 이탈률도 높아지는 경향이 있기 때문입니다.


## < 가입 월별 지역별 M+1 재접속률 >

In [ ]:
#@title Make DataFrame-1

## << auChurnedRateByRegionDf: 가입 연월별 지역별 - 최초 유입시점 볼륨 대비 미접속률 >>
query_job = client.query("""

  -- 일간 접속유저 테이블(+ 거주지역, 가입시점) 
  with active_user as (
    select a.login_date, a.login_year_month, b.create_date, b.create_year_month, a.user_id, b.region_1depth_name, b.region_2depth_name, region_3depth_name
    from `crested-drive-372522.my_temp.daily_login_users` as a
    inner join `crested-drive-372522.my_temp.all_myinfos` as b
    on a.user_id = b.user_id
  ),

  -- 가입월별, 접속월별, 지역별 접속 유저수 & 가입시점 기준 최초 접속 유저수 테이블
  active_user_by_create_month as (
     select login_year_month, create_year_month, region, user_cnt
          , first_value(user_cnt) over(partition by create_year_month, region order by login_year_month) as first_user_cnt
     from (
          select login_year_month, create_year_month, region_3depth_name as region 
               , count(distinct user_id) as user_cnt
          from active_user
          group by login_year_month, create_year_month, region
     ) as a
     order by create_year_month, login_year_month, region
  )

  select login_year_month, create_year_month, substr(create_year_month, 1, 4) as create_year, region
       , concat('M+', dense_rank() over(partition by create_year_month order by login_year_month) - 1) as time_order
       , user_cnt, first_user_cnt, round(user_cnt/first_user_cnt, 4) as rate_of_change
  from active_user_by_create_month 
""")
auChurnedRateByRegionDf = query_job.to_dataframe()
auChurnedRateByRegionDf = auChurnedRateByRegionDf.rename(columns={"first_user_cnt":"M+0_user_count"})


##---< 시각화 편의를 위한 데이터프레임 형태 변경 >-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# << auChurnedCntAndRateDf: 가입 연월별 지역별 - 월간(접속 유저수, 재접속률) - M+0부터 추적 가능한 2022-05 유입 유저부터 잘라냄 >>
auChurnedCntAndRateByRegionDf = auChurnedRateByRegionDf[auChurnedRateByRegionDf['create_year_month'] >= '2022-05']\
.sort_values(by=["create_year_month", "login_year_month", "region"])\
.melt(id_vars=["create_year_month", "login_year_month", "create_year", "region", "time_order"], value_vars=["user_cnt", "rate_of_change"]) 

In [ ]:
#@title Visualization-1

## << 가입연월별 최초유입시점 대비 M+1 재접속률 변화 >>
import plotly.express as px

## Step.1 - fig에 px.line() 그래프 담기
fig = px.line(auChurnedCntAndRateByRegionDf[(auChurnedCntAndRateByRegionDf['create_year_month'] <= '2022-10') &
                                            (auChurnedCntAndRateByRegionDf['create_year_month'] >= '2022-05') & 
                                            (auChurnedCntAndRateByRegionDf['time_order'].str.contains("0|1"))], # 비교에 필요한 부분만 잘라내는 과정
              x="time_order", 
              y="value", 
              color="region",
              facet_col = "create_year_month",
              # facet_col_spacing = 0.05, #  (float between 0 and 1) – Spacing between facet rows, in paper units. Default is 0.03 or 0.0.7 when facet_col_wrap is used.
              facet_row = "variable",
              markers=True,
              category_orders={"create_year_month": ["2022-05", "2022-06", "2022-07", "2022-08", "2022-09", "2022-10"],
                               "time_order": ["M+0", "M+1"]},
              )

## Step.2 - fig의 layout 설정 변경
fig.update_layout(
    {
        "title": {
            "text": "<b>< [별첨] 가입 연월별 지역별 AU의 M+1 재접속률 비교 ><b>",
            "x": 0.5,
            "y": 0.99,
            "font": {
                "size": 15
            }
        },
        "legend_traceorder": "normal", #normal(default - 데이터 순서대로), reversed(역순)
        "legend_title_text": "거주 지역"
    },
)

## Step.3 - fig의 y축을 기준으로, 직사각형 영역 표시 또는 텍스트를 남깁니다.
fig.add_vrect(x0="M+0", 
              x1="M+1",
              y0=0,
              y1=0.4,
              annotation_text="* 재접속률 0 ~ 50% 구간", 
              annotation_position="bottom left",
              fillcolor="hotpink", opacity=0.15, line_width=0,
              row=1) # https://plotly.com/python/horizontal-vertical-shapes/#horizontal-and-vertical-lines-in-dash 


## Step.4 - fig의 x축 또는 y축 상세 설정
fig.update_xaxes(title='', row=1)

fig.update_yaxes(title='활성 유저수(명)', row=2, col=1)
fig.update_yaxes(tickformat = ',', row=2)
fig.update_yaxes(range=[-100, 700], row=2)

fig.update_yaxes(title='재접속률(%)', row=1, col=1)
fig.update_yaxes(tickformat = '.0%', row=1)
fig.update_yaxes(tickformat = '.0%', row=1)
fig.update_yaxes(matches=None, showticklabels=True)
fig.update_yaxes(range=[0, 1.2], row=1)

## Step.5 - 그래프 출력
fig.show()

> **Comment**

- **이탈률 가속 현상은 다수의 지역에서 공통적으로 발생하고 있음**
  - 22년 10월에 모객된 AU 그룹의 M+1 재접속률을 거주지역 별로 살펴보면, 이전 모객그룹(5월~9월)에 비해 50% 이하로 떨어지는 지역이 다수 발생
  - 특히 모객 볼륨이 100명 이상인 목동(479명), 문래동(134명)의 경우 M+1 재접속률이 각각 34%, 27%로 전월比 7~8%p 하락
- **11월 이전 신규 모객 유저들의 이탈률 가속현상이 12월에도 관찰되는지 추적 관찰이 필요하며, 이탈 원인에 대해서는 추후 별도의 분석 진행 예정**

> **Tip**

- 한 화면에서 표현되는 정보의 양이 많을수록 어느 부분을 중점으로 살펴봐야하는지 표시해 주는게 좋습니다.
- 현황 분석시 특정 이슈의 원인을 끝까지 파고드는것 보다는, 최대 1~2 Depth 까지만 추가로 살펴보고 그 이상의 분석은 별도의 분석으로 분리해서 가져가는 것을 추천합니다. 

# **[ 잔존율 분석 ]**

## < 요약 ✨ >

> 작성 필요

In [ ]:
#@title Make DataFrame

## << monthlyRetByUserType: 유저 타입별 월간 일평균 D+day 잔존율 >>
query_job = client.query("""
    with monthly_new_user_list as (
        select create_year_month
             , create_date
             , user_id
        FROM `my_temp.all_myinfos`
    ), 

    daily_login as (
        select distinct login_date
             , login_year_month
             , user_id
        from `my_temp.daily_login_users`
    ),

    d_plus_day_check_login as (
        select distinct a.login_date, a.login_year_month
             , date_diff(CAST('2022-11-30' AS DATE), a.login_date, DAY)  as date_diff_count
             , a.user_id
             , case when d1.user_id is not null then 1 end as d1_login
             , case when d7.user_id is not null then 1 end as d7_login
             , case when d14.user_id is not null then 1 end as d14_login
             , case when d30.user_id is not null then 1 end as d30_login
        from (
          select login_date, login_year_month, user_id
          from daily_login
        ) as a 
        left join (
          select login_date, user_id 
          from daily_login
        ) as d1
        on a.user_id = d1.user_id and date_add(a.login_date, interval 1 day) = d1.login_date
        left join (
          select login_date, user_id 
          from daily_login
        ) as d7
        on a.user_id = d7.user_id and date_add(a.login_date, interval 7 day) = d7.login_date
        left join (
          select login_date, user_id 
          from daily_login
        ) as d14
        on a.user_id = d14.user_id and date_add(a.login_date, interval 14 day) = d14.login_date
        left join (
          select login_date, user_id 
          from daily_login
        ) as d30
        on a.user_id = d30.user_id and date_add(a.login_date, interval 30 day) = d30.login_date
        order by a.login_date, a.user_id
    )

    select login_year_month
         , case when user_type is null then '전체' else user_type end as user_type
         , count(distinct user_id) as mau
         , count(user_id) as mau_not_unique
        -- 신규유저는 하나의 행만 가지고 있으므로 유저수 집계시 count(distinct user_id)를 사용해도 되지만, 재방문 유저는 기간내에 여러번 방문 가능하기 때문에 count(user_id)를 사용해야한다. ⭐️
        --  , count(case when date_diff_count >= 1 then user_id end) as mau_not_unique_d1
        --  , count(case when date_diff_count >= 7 then user_id end) as mau_not_unique_d7
        --  , count(case when date_diff_count >= 14 then user_id end) as mau_not_unique_d14
        --  , count(case when date_diff_count >= 30 then user_id end) as mau_not_unique_d30
         , round(safe_divide(sum(case when date_diff_count >= 1 then d1_login end), count(case when date_diff_count >= 1 then user_id end)), 3) as d1_ret
         , round(safe_divide(sum(case when date_diff_count >= 7 then d7_login end), count(case when date_diff_count >= 7 then user_id end)), 3) as d7_ret
         , round(safe_divide(sum(case when date_diff_count >= 14 then d14_login end), count(case when date_diff_count >= 14 then user_id end)), 3) as d14_ret
         , round(safe_divide(sum(case when date_diff_count >= 30 then d30_login end), count(case when date_diff_count >= 30 then user_id end)), 3) as d30_ret
    from (
      select a.login_date, a.login_year_month, date_diff_count
             , a.user_id
             , d1_login
             , d7_login
             , d14_login
             , d30_login
             , case when b.user_id is not null then '신규유저' else '재방문유저' end as user_type
      from d_plus_day_check_login as a
      left join monthly_new_user_list as b
      on a.user_id = b.user_id and a.login_date = b.create_date
    ) as a
    group by rollup(a.login_year_month, a.user_type)
    having a.login_year_month is not null
    order by login_year_month, user_type
""")
monthlyRetByUserType = query_job.to_dataframe()


## << dauNdayRet: 유저 타입별 일간 D+day 잔존율 >>
query_job = client.query("""
    with monthly_new_user_list as (
        select create_year_month
             , create_date
             , user_id
        FROM `my_temp.all_myinfos`
    ), 

    daily_login as (
        select distinct login_date
             , login_year_month
             , user_id
        from `my_temp.daily_login_users`
    ),

    d_plus_day_check_login as (
        select distinct a.login_date, a.login_year_month
             , date_diff(CAST('2022-11-30' AS DATE), a.login_date, DAY)  as date_diff_count
             , a.user_id
             , case when d1.user_id is not null then 1 end as d1_login
             , case when d7.user_id is not null then 1 end as d7_login
             , case when d14.user_id is not null then 1 end as d14_login
             , case when d30.user_id is not null then 1 end as d30_login
        from (
          select login_date, login_year_month, user_id
          from daily_login
        ) as a 
        left join (
          select login_date, user_id 
          from daily_login
        ) as d1
        on a.user_id = d1.user_id and date_add(a.login_date, interval 1 day) = d1.login_date
        left join (
          select login_date, user_id 
          from daily_login
        ) as d7
        on a.user_id = d7.user_id and date_add(a.login_date, interval 7 day) = d7.login_date
        left join (
          select login_date, user_id 
          from daily_login
        ) as d14
        on a.user_id = d14.user_id and date_add(a.login_date, interval 14 day) = d14.login_date
        left join (
          select login_date, user_id 
          from daily_login
        ) as d30
        on a.user_id = d30.user_id and date_add(a.login_date, interval 30 day) = d30.login_date
        order by a.login_date, a.user_id
    )

    select login_date
         , case when user_type is null then '전체' else user_type end as user_type
         , count(distinct user_id) as mau
         , count(user_id) as mau_not_unique
         , count(distinct login_date) as day_cnt
         , count(distinct user_id) as dau
         , count(user_id) as dau_not_unique
         , round(sum(d1_login)/count(user_id), 3) as d1_ret
         , round(sum(d7_login)/count(user_id), 3) as d7_ret
         , round(sum(d14_login)/count(user_id), 3) as d14_ret
         , round(sum(d30_login)/count(user_id), 3) as d30_ret
    from (
      select a.login_date, a.login_year_month, date_diff_count
             , a.user_id
             , d1_login
             , d7_login
             , d14_login
             , d30_login
             , case when b.user_id is not null then '신규유저' else '재방문유저' end as user_type
      from d_plus_day_check_login as a
      left join monthly_new_user_list as b
      on a.user_id = b.user_id and a.login_date = b.create_date
    ) as a
    group by rollup(a.login_date, a.user_type)
    having a.login_date is not null
    order by login_date, user_type
""")
dauNdayRet = query_job.to_dataframe()


# import pickle

# with open( "/content/drive/MyDrive/ordersDf", "wb" ) as file:
#     pickle.dump(ordersDf, file)

# with open("/content/drive/MyDrive/ordersDf", "rb" ) as file:
#     ordersDfFromPK = pickle.load(file)

In [ ]:
#@title Visualization-1 

## << 신규 유저 월간 일평균 N-day 잔존율 >>

import plotly.express as px
from plotly.subplots import make_subplots

# step.1 - 전체 유저의 잔종율 데이터만 가져오기
newMauNdayRet = monthlyRetByUserType[monthlyRetByUserType['user_type'] == '신규유저'] # (전체/신규유저/재방문유저)

# step.2 - 서브플랏 할당
subfig = make_subplots(specs=[[{"secondary_y": True}]])

# step.3 - fig1(잔존율 그래프) 생성
fig1 = px.line(newMauNdayRet, 
              x="login_year_month", 
              y=['d1_ret', 'd7_ret', 'd14_ret', 'd30_ret'],
              line_group="variable", 
              text="value",
              markers=True,
             )

fig1.update_traces(
    yaxis="y2",
    textposition='top center',
    texttemplate='%{text: .1%}',
    hovertemplate = '%{y:.1%}',
)

# step.4 - fig2(mau 그래프) 생성
fig2 = px.bar(newMauNdayRet, 
             x="login_year_month", 
             y="mau", 
            )
fig2.update_traces(marker_color='lightgray')  # e.g. red, hotpink, blue, skyblue, ...

# step.5 - subfig에 fig1, fig2를 통합 할당
subfig.add_traces(fig1.data + fig2.data)
subfig.update_layout(
    {
        "title": {
            "text": "<b>< 신규 유저 월간 일평균 N-day 잔존율 ><b>",
            "x": 0.5,
            "y": 0.9,
            "font": {
                "size": 13
            }
        },
        "xaxis": {
            "title": "",
            "tick0": "2022-05-01",
            "dtick": 'M1',
            "tickformat": "%m월"
        },
        "yaxis": { "title": "MAU(명)", "tickformat": ',' },
        "yaxis2": { "title": "Avg. N-day Retention(%)", "tickformat": '.1%', "range": [0, 0.8]},
        "legend_title_text": "N-day 잔존율",
        "template":'plotly_white',
    }
)

subfig.update_layout(hovermode="x")

# step.6 - secondary_y 축의 gird 만 보이게 하기
subfig['layout']['yaxis1']['showgrid'] = False

# step.7 - 그래프 출력
subfig.show()

> **Note**

- 11월 잔존율의 경우 집계 가능한 모수로 한정하여 평균값이 계산됨 (예를들어 D+14 잔존율은 (11/1~11/16) 접속유저에 대한 D+14 잔존율 평균값을 제공)
- 따라서 11월 D+7 ~ D+14 일평균 잔존율의 경우 전체 모수에 대한 집계값이 아니므로 참고용으로만 사용

> **Comment**

- 작성 필요

In [ ]:
#@title Visualization-2

## << 신규 유저 월간 일평균 N-day 잔존율 >>

import plotly.express as px
from plotly.subplots import make_subplots

# step.1 - 전체 유저의 잔존율 데이터만 가져오기
newDauNdayRet = dauNdayRet[dauNdayRet['user_type'] == '신규유저'] # (전체/신규유저/재방문유저)

# step.2 - 서브플랏 할당
subfig = make_subplots(specs=[[{"secondary_y": True}]])

# step.3 - fig1(잔존율 그래프) 생성
fig1 = px.line(newDauNdayRet, 
              x="login_date", 
              y=['d1_ret', 'd7_ret', 'd14_ret', 'd30_ret'],
              color="variable", 
             )
fig1.update_traces(yaxis="y2") # fig1의 y축을 보조축으로 변환

# step.4 - fig2(dau 그래프) 생성
fig2 = px.bar(newDauNdayRet, 
             x="login_date", 
             y="dau", 
            )
fig2.update_traces(marker_color='lightgray')

# step.5 - subfig에 fig1, fig2를 통합 할당
subfig.add_traces(fig1.data + fig2.data)
subfig.update_layout(
    {
        "title": {
            "text": "<b>< 신규 유저 N-day 잔존율 ><b>",
            "x": 0.5,
            "y": 0.8,
            "font": {
                "size": 13
            }
        },
        "xaxis1": {
            "title": "",
            "tick0": "2022-05-01",
            "dtick": 'M1',
            "tickformat": "%Y-%m-%d"
        },
        "yaxis": { "title": "DAU(명)", "tickformat": ',', "range": [0, 1000] },
        "yaxis2": { "title": "N-day Retention(%)", "tickformat": '.0%', "range": [0, 1]},
        "legend_title_text": "N-day 잔존율",
        "template":'plotly_white',
    }
)

# step.6 - 잔존율 탐색 기준선 도식
subfig.add_hline(y=0.7, 
                 line_dash="dot", 
                 secondary_y=True,
                 )
# step.6 - 잔존율 탐색 기준선 관련 내용 표기
subfig.add_annotation(
    x='2022-11-07',
    y=700,
    xref="x",
    yref="y",
    text="잔존율 70% 기준선",
    showarrow=True,
    font=dict(
        size=12,
        color="Gray"
    ),
    align="right",
    arrowhead=2,
    ax=-60,
    ay=-30
)


# step.7 - 신규 매장 입점 기간 표시
subfig.add_vrect(x0="2022-06-13", 
                 x1="2022-06-16",
                 annotation_text="* 신규 매장 입점(6/14 ~ 6/15)", 
                 annotation_position="top left",
                 fillcolor="skyblue", opacity=0.25, line_width=0)

subfig.add_vrect(x0="2022-11-06", 
                 x1="2022-11-09",
                 annotation_text="* 신규 매장 입점(11/7 ~ 11/8)", 
                 annotation_position="top right",
                 fillcolor="skyblue", opacity=0.25, line_width=0)

subfig.add_vrect(x0="2022-11-21", 
                 x1="2022-11-30",
                 annotation_text="* 11월 3주차 ~", 
                 annotation_position="top right",
                 fillcolor="hotpink", opacity=0.25, line_width=0)

# step.8 - secondary_y 축의 gird 만 보이게 하기
subfig['layout']['yaxis1']['showgrid'] = False

# step.9 - 그래프 출력
subfig.show()

> **Comment**

- 작성 필요


> **Tip**

- 월간 지표를 보았다면 보조 지표로 일간 지표도 같이 확인해 주는게 좋습니다. 
- 특히 평균을 계산하는 경우 특이값의 영향을 많이 받기 때문에 기간을 다르게 설정할 경우 전혀 다른 수치가 계산될 수 있습니다.